In [43]:
import numpy as np
from typing import Tuple
from enum import Enum

In [52]:
class Pivoting(Enum):
  NO_PIVOTING = 0,
  PARTIAL_PIVOTING = 1,
  FULL_PIVOTING = 2

def do_partial_pivoting(A: np.ndarray, b:np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
  # find hihest row value
  index_highest_values_per_column = np.abs(A).argmax(1)
  index_hihest_first_column = index_highest_values_per_column[0]

  # Swap rows
  A[[0, index_hihest_first_column]] = A[[index_hihest_first_column, 0]]
  b[[0, index_hihest_first_column]] = b[[index_hihest_first_column, 0]]

  return (A, b)

def do_full_pivoting(A : np.ndarray, b:np.ndarray) -> Tuple[np.ndarray, np.ndarray, int]:
  # Find the highest value
  highest_value = max(A.min(), A.max(), key=abs)
  max_value = np.where(A == highest_value)

  # Use the first found occurence if there is more than one
  index_row_highest_value = max_value[0][0]
  index_column_highest_value = max_value[1][0]

  # Swap rows
  A[[0, index_row_highest_value]] = A[[index_row_highest_value, 0]]
  b[[0, index_row_highest_value]] = b[[index_row_highest_value, 0]]

  # Swap columns
  A[:, [0, index_column_highest_value]] = A[:, [index_column_highest_value, 0]]

  return (A, b, index_column_highest_value)

# Gaussian Elimination method

This method for solving linear systems works by forming a superior triangle matrix 

In [61]:
def gauss_elimination(A: np.ndarray, b:np.ndarray, pivoting = Pivoting.NO_PIVOTING) -> np.ndarray:
  assert len(A) == len(b)
  assert A.shape[0] == A.shape[1]
  exchanged_columns = []

  for index_col in range(len(A)):
    match pivoting:
      case Pivoting.NO_PIVOTING:
        pass
      case Pivoting.PARTIAL_PIVOTING:
        A[index_col:, index_col:] , b[index_col:] = do_partial_pivoting(A[index_col:, index_col:] , b[index_col:])
      case Pivoting.FULL_PIVOTING:
        A[index_col:, index_col:], b[index_col:], column_excanged = do_full_pivoting(A[index_col:, index_col:] , b[index_col:])
        exchanged_columns.append(column_excanged)
        # raise NotImplementedError("Not implemented function yet")
      
    pivot = A[index_col][index_col]
    for index_row_below in range(index_col + 1, len(A)):
      ratio = A[index_row_below][index_col] / pivot
      
      for idx in range(index_col, len(A)):
        A[index_row_below][idx] -= A[index_col][idx] * ratio

      b[index_row_below] -= b[index_col] * ratio
      

  result = np.linalg.solve(A, b)
  if pivoting == Pivoting.FULL_PIVOTING:
    exchanged_columns.reverse()
    for index, excahnged in enumerate(exchanged_columns):
      result[[index, excahnged]] = result[[index, excahnged]]

  return result

# Gauss with pivoting

In [46]:
A = np.array([[1,-7], [5,2]], dtype=np.float64)
b = np.array([-11,-18], dtype=np.float64)
# Solution x=-4, y=1
gauss_elimination(A,b, Pivoting.PARTIAL_PIVOTING)

array([-4.,  1.])

In [57]:
# 1x + 2y - 2z = -15
# 2x + 1y - 5z = -21
# 1x - 4y + 1z = +18 
# Solution {-1, -4, 3}

A = np.array([
  [1, 2, -2],
  [2, 1, -5],
  [1, -4, 1]
], dtype=np.float64)

b = np.array([-15, -21, 18], dtype=np.float64)

gauss_elimination(A, b, Pivoting.FULL_PIVOTING)

array([-1., -4.,  3.])

In [62]:
A = np.array([
  [1.,    2.,   1., -1.],
  [3./2., 1.,   2., 2.],
  [4.,    4.,   3., 4.],
  [2./5., 0.,  1./5., 1.]
], dtype=np.float64)

b = np.array([5., 8., 22., 3.], dtype=np.float64)

# x = 16
# y = -6
# z = -2
# t = -3

gauss_elimination(A, b, Pivoting.FULL_PIVOTING)

array([-2., -3., 16., -6.])